# Case Study Evaluation 

This notebook uses exploratory data visualization and statistical models to evaluate the viability of the selected case study and the driver data provided. This notebook will use the data inputs and directories generated by the 1_data_acquisition_format notebook. 

You can use the visualizations and model results to assess:

* Whether the commodity pathways explain the observed introductions
* How prominent to expect bridgehead introductions to be
* How to expect to weight establishment vs. entry


In [ ]:
# Workspace 
import os
import glob
import dotenv

# Data manipulation
import pandas as pd
import numpy as np
import geopandas as gpd
from functools import reduce

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# GLM 
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

# Random Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.inspection import permutation_importance

## Set Environmental Variables and Paths

In [ ]:
# If notebook was launched from notebook folder of the clone GitHub
# repository, then set working directory to level above
# (e.g., '..' to navigate to /PoPS-Global)

# This should be the path where the .env file is saved
repo_path = str(input())
os.chdir(repo_path)
print(os.getcwd())

In [ ]:
# Load variables and paths from .env
dotenv.load_dotenv('.env')

In [ ]:
# Path to case-study specific raw data inputs (host map, phytosanitary capacity dataset)
data_dir = os.getenv("DATA_PATH")

# Path to formatted model inputs
input_dir = os.getenv("INPUT_PATH")

# Path to save outputs
out_dir = os.getenv("OUTPUT_PATH")

## Import formatted data

In [ ]:
# Read in the all driver data and create a static tabular format (row = country, column = feature)
# Country dataframe, host area, phytosanitary capacity
countries_gdf = gpd.read_file(glob.glob(input_dir + "countries*.gpkg")[0])

# Validation data
validation = pd.read_csv(glob.glob(input_dir +'*first_records_validation.csv')[0])
validation = validation.loc[validation['ISO3'].isin(countries_gdf['ISO3'])]

# Origin data
origins = pd.read_csv(input_dir + 'origin_locations.csv')
origins = origins.loc[origins['ISO3'].isin(countries_gdf['ISO3'])]


In [ ]:
# Binary of origins
countries_gdf.loc[countries_gdf['ISO3'].isin(origins.ISO3),'Origin'] = 1
countries_gdf.loc[~countries_gdf['ISO3'].isin(origins.ISO3),'Origin'] = 0

# Binary of destinations
countries_gdf.loc[countries_gdf['ISO3'].isin(validation.ISO3),'Destination'] = 1
countries_gdf.loc[~countries_gdf['ISO3'].isin(validation.ISO3),'Destination'] = 0


In [ ]:
# Climate similarities matrix
climate_similarities = np.load(glob.glob(input_dir + "climate_similarities*.npy")[0])


In [ ]:
# For each country, what is the max similarity to all pest origin locations?
countries_gdf['Climate_Max'] = np.max(climate_similarities[countries_gdf['Origin']==1,], axis=0)

# Option to run mean similarity:
countries_gdf['Climate_Mean'] = np.mean(climate_similarities[countries_gdf['Origin']==1], axis=0)

In [ ]:
# Distance matrix
distance = np.load(glob.glob(input_dir + "distance_matrix*.npy")[0])

# Set the diagonal (self-distances) to NaN
np.fill_diagonal(distance, np.nan)


In [ ]:
# For each country, what is the minimum distance to a pest origin or bridgehead location?
countries_gdf['Dist_Origin'] = np.nanmin(distance[countries_gdf['Origin']==1], axis=0)
countries_gdf['Dist_Origin'] = MinMaxScaler().fit_transform(np.array(countries_gdf['Dist_Origin']).reshape(-1,1))

countries_gdf['Dist_Bridge'] = np.nanmin(distance[countries_gdf['Destination']==1], axis=0)
countries_gdf['Dist_Bridge'] = MinMaxScaler().fit_transform(np.array(countries_gdf['Dist_Bridge']).reshape(-1,1))

In [ ]:
# Trade - two columns per commodity

# Total cumulative imports from (1) all source countries
# Total cumulative imports from (2) all validation countries

In [ ]:
# Range of time for validation data
start_year = validation['ObsFirstIntro'].min() - 5
end_year = validation['ObsFirstIntro'].max()

# If these are beyond the bounds of your trade data, overwrite with static years

In [ ]:
# Total trade data by commodity during the period of interest
# From source countries
# From validation countries (ie. potential bridgeheads)

year_range = list(range(start_year, end_year + 1, 1))
commodities = os.listdir(input_dir + 'comtrade/monthly_adjusted/')

for commodity in commodities: 
    
    try:
        del trade_sum
    except:
        print("Initializing...")
    for d in year_range:
        d_file_list = glob.glob(input_dir + f'/comtrade/monthly_adjusted/{commodity}/*_{d}*.csv')
        print(f'{commodity}, {d}: {len(d_file_list)}')
        dfs = [pd.read_csv(f, sep = ",", header= 0, index_col=0, encoding='latin1') for f in d_file_list]
        all_com = reduce(pd.DataFrame.add, dfs)
        try:
            trade_sum += all_com
            print('Added to trade_sum')
        except:
            trade_sum = all_com
            print('Created trade_sum')
    
    # Keep only origin exporters and other validation countries (bridgeheads)
    countries_gdf[f'Origin_{commodity}'] = trade_sum[origins.ISO3].sum(axis=1).reset_index(drop=True)
    countries_gdf[f'Origin_{commodity}'] = MinMaxScaler().fit_transform(np.array(countries_gdf[f'Origin_{commodity}']).reshape(-1,1))
    countries_gdf[f'Bridge_{commodity}'] = trade_sum[validation.ISO3].sum(axis=1).reset_index(drop=True)
    countries_gdf[f'Bridge_{commodity}'] = MinMaxScaler().fit_transform(np.array(countries_gdf[f'Bridge_{commodity}']).reshape(-1,1))
    print(f'{commodity} summed.\n')



In [ ]:
# Keep host area and phytosanitary capacity as is
countries_gdf['Host_Area'] = countries_gdf['Host Percent Area']
countries_gdf['P_Cap'] = countries_gdf['Phytosanitary Capacity']


In [ ]:
# Create simplified dataframe
regression_df = countries_gdf.set_index('ISO3')
regression_df = regression_df.loc[:,'Origin':].reset_index()

In [ ]:
# Write out to .csv for future use
regression_df.to_csv(input_dir + "regression_data.csv", index=False)


## Exploratory visualization

In [ ]:
# If you've already done the setup, read back in the regression data 
regression_df = pd.read_csv(input_dir + "noTWN/regression_data.csv")

# Origin countries are excluded from this analysis
regression_data = regression_df.loc[regression_df['Origin']==0].reset_index(drop=True).drop(columns="Origin")


In [ ]:
# Visualize the data
sns.pairplot(regression_data, hue = "Destination")
plt.show()


Points for interpretation: 

- Do the introduced locations separate clearly from the rest of the data on any of the pathway variables (ie. traded commodities)? (esp. high values)
- Does the relationship appear different with trade from origin vs. bridgeheads?
- Is there a clear relationship with distance, indicating a role of cross-border spread or natural dispersal? (esp. low values)


## Model analysis: Binomial regression

In [ ]:
# Create the regression formula
predictors = regression_data.loc[:,'Climate_Max':'P_Cap'].columns
predictor_string = ' + '.join(predictors)

print(f'Regression formula: Destination ~ {predictor_string}')


In [ ]:
# Set up data matrices
y, X = dmatrices(f'Destination ~ {predictor_string}', data=regression_data, return_type='dataframe')


In [ ]:
# Run binomial regression model for binary presence outcome
binomial_model = sm.GLM(y, X, family=sm.families.Binomial())
binomial_results = binomial_model.fit()
print(binomial_results.summary())


Points for interpretation:

- Because the PoPS Global simulation model treats drivers mechanistically, predictors should *positively* correlate with pest introduction, EXCEPT for distance 
- Consider removing commodity pathways with negative values
- Strong negative correlation with distance may suggest a role for natural dispersal or cross border transportation beyond what is captured through trade
- Highly predictive variables may cause "Perfect separation" error - if this is the case, you can use the [RJAGS version of the model](https://github.com/arielsaffer/pandemic-statespace)
- Be cautious on interpretation due to the presence of confounding variables. Note that as this is a static treatment of trade volume (cumulative), the presence of high volume events or changes to volume over time will be missed.

## Model analysis: Random Forest (all predictors)

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y.Destination.values, test_size=0.3) 

# Fit the model
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

# Predict to assess accuracy
y_pred=clf.predict(X_test)
print(f"Accuracy score: {metrics.accuracy_score(y_test, y_pred)}; \nReal positives: {y_test.sum()} (Predicted as: {y_pred[y_test==1]}) \nPredicted positives: {y_pred.sum()} (Real values: {y_test[y_pred==1]})")

# Be wary that for case studies with few known introductions, this model may be "accurate" by producing all 0's

In [ ]:
# Assess feature importance
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

In [ ]:
# Visualize feature importance - within the original model 
sns.barplot(x=feature_imp, y=feature_imp.index, palette="mako")
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("All Important Features")
plt.show()

In [ ]:
# Visualize permutation importance within the test data
result = permutation_importance(
    clf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=X_test.columns[sorted_idx]
)
ax.set_title("Permutation Importances")
fig.tight_layout()
plt.show()

Points for interpretation:

- Importance is relevant, BUT, we don't see here if the relationship is positive or negative
- - E.g. countries importing tomatoes may be LESS likely to have the virus, because they produce fewer tomatoes
- To help with this, try the RF with preselected predictors below, informed by the binomial regression

## Model analysis: Random Forest (pre-selected predictors)

In [ ]:
# Select only positive pathways/negative distance predictors

# Positive predictors - host, area,
positive_predictors = list(binomial_results.params.index[binomial_results.params>0])

# Except for distance which is relevant if negative
for col in ['Dist_Origin','Dist_Bridge']:
    if binomial_results.params[col] < 0:
        positive_predictors.append(col)

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X[positive_predictors], y.Destination.values, test_size=0.3) 

# Fit the model
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

# Predict to assess accuracy
y_pred=clf.predict(X_test)
print(f"Accuracy score: {metrics.accuracy_score(y_test, y_pred)}; \nReal positives: {y_test.sum()} (Predicted as: {y_pred[y_test==1]}) \nPredicted positives: {y_pred.sum()} (Real values: {y_test[y_pred==1]})")

In [ ]:
# Assess feature importance
feature_imp = pd.Series(clf.feature_importances_,index=X[positive_predictors].columns).sort_values(ascending=False)
feature_imp

In [ ]:
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index, palette="mako")
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Important Features \nCommodities and Host/Climate +; Distance - ")
plt.show()

In [ ]:
result = permutation_importance(
    clf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=X_test.columns[sorted_idx]
)
ax.set_title("Permutation Importances")
fig.tight_layout()
plt.show()

Based on the data visualization and outcomes of the models, determine if the pathway drivers (traded commodities) are appropriate predictors of pest transport. If additional sources are needed, acquire with notebook 1 and re-run this analysis. 

If commodities do not appear to positively drive pest presence, consider removing them, especially if you will be aggregating multiple commodities.

Once you have selected reasonable drivers, proceed to model calibration.